<a href="https://colab.research.google.com/github/deva41103/StockGPT/blob/main/StockGpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf

# Example: Fetch 5 years of daily data for RELIANCE
data = yf.download("RELIANCE.NS", start="2018-01-01", end="2025-07-30")


/tmp/ipython-input-585614236.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("RELIANCE.NS", start="2018-01-01", end="2025-07-30")
[*********************100%***********************]  1 of 1 completed


In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler()
data['Close'] = scaler.fit_transform(data[['Close']])

def create_dataset(series, lookback=60, forecast_horizon=1):
    X, y = [], []
    for i in range(len(series) - lookback - forecast_horizon):
        X.append(series[i:i + lookback])
        y.append(series[i + lookback + forecast_horizon - 1])
    return np.array(X), np.array(y)

# For 1-day, 7-day, 30-day prediction
X_1d, y_1d = create_dataset(data['Close'].values, forecast_horizon=1)
X_7d, y_7d = create_dataset(data['Close'].values, forecast_horizon=7)
X_30d, y_30d = create_dataset(data['Close'].values, forecast_horizon=30)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

def build_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(50),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

model_1d = build_model((X_1d.shape[1], 1))
model_1d.fit(X_1d.reshape(-1, X_1d.shape[1], 1), y_1d, epochs=20, batch_size=32)


Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 8s 48ms/step - loss: 0.0930
Epoch 2/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - loss: 0.0039
Epoch 3/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - loss: 0.0036
Epoch 4/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - loss: 0.0033
Epoch 5/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - loss: 0.0032
Epoch 6/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0035
Epoch 7/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - loss: 0.0031
Epoch 8/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - loss: 0.0028
Epoch 9/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 6s 70ms/step - loss: 0.0029
Epoch 10/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0029
Epoch 11/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0030
Epoch 12/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - loss: 0.0029
Epoch 13/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 71ms/step - loss: 0.0027
Epoch 14/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0023
Epoch 15/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0025
Epoch 16/20
57/

In [ ]:
last_60 = data['Close'].values[-60:].reshape(1, 60, 1)
pred_1d = model_1d.predict(last_60)
predicted_price = scaler.inverse_transform(pred_1d)[0][0]
print("Predicted price (1-day ahead):", predicted_price)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 570ms/step
Predicted price (1-day ahead): 1440.6809


In [ ]:
import yfinance as yf
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

# Create folder
os.makedirs("models", exist_ok=True)

def create_dataset(series, lookback=60, forecast_horizon=1):
    X, y = [], []
    for i in range(len(series) - lookback - forecast_horizon):
        X.append(series[i:i + lookback])
        y.append(series[i + lookback + forecast_horizon - 1])
    return np.array(X), np.array(y)

def build_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(50),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Train for RELIANCE.NS
symbol = "RELIANCE.NS"
df = yf.download(symbol, start="2018-01-01", end="2025-07-31")
data = df['Close'].values.reshape(-1, 1)

scaler = MinMaxScaler()
scaled = scaler.fit_transform(data)

intervals = {'1d': 1, '1w': 7, '1m': 30}
for label, horizon in intervals.items():
    X, y = create_dataset(scaled, 60, forecast_horizon=horizon)
    X = X.reshape(X.shape[0], X.shape[1], 1)
    model = build_model((60, 1))
    model.fit(X, y, epochs=20, batch_size=32, verbose=1)
    model.save(f"models/lstm_{label}.h5")


/tmp/ipython-input-3124248155.py:31: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start="2018-01-01", end="2025-07-31")
[*********************100%***********************]  1 of 1 completed

Epoch 1/20



/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 6s 49ms/step - loss: 0.0742
Epoch 2/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 70ms/step - loss: 0.0048
Epoch 3/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 48ms/step - loss: 0.0038
Epoch 4/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 7s 87ms/step - loss: 0.0034
Epoch 5/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 66ms/step - loss: 0.0030
Epoch 6/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - loss: 0.0033
Epoch 7/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0034
Epoch 8/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 53ms/step - loss: 0.0030
Epoch 9/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0033
Epoch 10/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0029
Epoch 11/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - loss: 0.0027
Epoch 12/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - loss: 0.0027
Epoch 13/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0030
Epoch 14/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0025
Epoch 15/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - loss: 0.0027
Epoch 16/20
57/

Epoch 1/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - loss: 0.0585
Epoch 2/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0061
Epoch 3/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - loss: 0.0049
Epoch 4/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - loss: 0.0045
Epoch 5/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0048
Epoch 6/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 6s 70ms/step - loss: 0.0039
Epoch 7/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0041
Epoch 8/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0039
Epoch 9/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 6s 56ms/step - loss: 0.0038
Epoch 10/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - loss: 0.0040
Epoch 11/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0036
Epoch 12/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0036
Epoch 13/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 6s 70ms/step - loss: 0.0038
Epoch 14/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0038
Epoch 15/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0036
Epoc

Epoch 1/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 6s 48ms/step - loss: 0.0718
Epoch 2/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0085
Epoch 3/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - loss: 0.0088
Epoch 4/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0087
Epoch 5/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0075
Epoch 6/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 6s 56ms/step - loss: 0.0073
Epoch 7/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0078
Epoch 8/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0079
Epoch 9/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - loss: 0.0084
Epoch 10/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 4s 65ms/step - loss: 0.0082
Epoch 11/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - loss: 0.0078
Epoch 12/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0073
Epoch 13/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0078
Epoch 14/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 4s 71ms/step - loss: 0.0077
Epoch 15/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - loss: 0.0075
Epoc

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError # Import the specific loss function
import numpy as np

# Load the models, providing custom objects for built-in functions
model_1d = load_model("models/lstm_1d.h5", custom_objects={'mse': MeanSquaredError()})
model_1w = load_model("models/lstm_1w.h5", custom_objects={'mse': MeanSquaredError()})
model_1m = load_model("models/lstm_1m.h5", custom_objects={'mse': MeanSquaredError()})


# Get the last 60 days of data and reshape for prediction
last_60_days = scaled[-60:].reshape(1, 60, 1)

# Make predictions for each horizon
pred_1d_scaled = model_1d.predict(last_60_days)
pred_1w_scaled = model_1w.predict(last_60_days)
pred_1m_scaled = model_1m.predict(last_60_days)

# Inverse transform the predictions to get actual prices
predicted_price_1d = scaler.inverse_transform(pred_1d_scaled)[0][0]
predicted_price_1w = scaler.inverse_transform(pred_1w_scaled)[0][0]
predicted_price_1m = scaler.inverse_transform(pred_1m_scaled)[0][0]

print("Predicted price (1-day ahead):", predicted_price_1d)
print("Predicted price (1-week ahead):", predicted_price_1w)
print("Predicted price (1-month ahead):", predicted_price_1m)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step
Predicted price (1-day ahead): 1407.0287
Predicted price (1-week ahead): 1400.7434
Predicted price (1-month ahead): 1379.9015
